<a href="https://colab.research.google.com/github/RealRaghu09/ChessAI.com/blob/main/ai-chess/finetuned_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q unsloth unsloth_zoo
!pip install -q --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 154.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.version.cuda)

True
1
12.8


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! Unsloth also supports RoPE (Rotary Positinal Embedding) scaling internally.
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit, # Will load the 4Bit Quantized Model
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # a higher alpha value assigns more weight to the LoRA activations
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2026.1.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
import pandas as pd
data_path = "/content/drive/MyDrive/Colab Notebooks/small_dataset.csv"
df = pd.read_csv(data_path)
df

,movetext,white_elo,black_elo,white_diff,black_diff,opening,eco
0,1. e4 e6 2. d4 b6 3. a3 Bb7 4. Nc3 Nh6 5. Bxh6...,1639.0,1403.0,5.0,-8.0,French Defense: Normal Variation,C00
1,1. d4 d5 2. Nf3 Nf6 3. e3 Bf5 4. Nh4 Bg6 5. Nx...,1654.0,1919.0,19.0,-22.0,"Queen's Pawn Game: Colle System, Anti-Colle",D04
2,1. e4 e5 2. Nf3 Nc6 3. Bc4 Nf6 4. Nc3 Bc5 5. a...,1643.0,1747.0,13.0,-94.0,Four Knights Game: Italian Variation,C50
3,1. e4 c6 2. Nc3 d5 3. Qf3 dxe4 4. Nxe4 Nd7 5. ...,1824.0,1973.0,-6.0,8.0,Caro-Kann Defense: Goldman Variation,B12
4,1. e4 e6 2. f4 d5 3. e5 c5 4. Nf3 Qb6 5. c3 Nc...,1765.0,1815.0,-9.0,9.0,French Defense: La Bourdonnais Variation,C00
...,...,...,...,...,...,...,...
3995,1. g3 Nf6 2. Bg2 e6 3. d4 d5 4. Nc3 c5 5. dxc5...,1382.0,1368.0,-12.0,16.0,Hungarian Opening: Indian Defense,A00
3996,1. d3 g6 2. e3 Bg7 3. d4 d6 4. g3 Nf6 5. Bg2 O...,1709.0,1401.0,4.0,-3.0,Mieses Opening,A00
3997,1. f4 e6 2. Nf3 c6 3. e3 b5 4. Be2 Bb7 5. O-O ...,1401.0,1649.0,-4.0,7.0,Bird Opening,A02
3998,1. e4 e5 2. Bc4 d6 3. Nf3 Nc6 4. d3 f6 5. O-O ...,1567.0,1262.0,-20.0,23.0,Philidor Defense #3,C41


In [ ]:
r1_prompt = """You are a strong chess engine. Given a chess position in FEN notation, predict the single best legal move for the current player in UCI format. Output only the move.
<movetext>
{}
</movetext>

<white_elo>
{}
</white_elo>

<black_elo>
{}
</black_elo>

<white_diff>
{}
</white_diff>

<black_diff>
{}
</black_diff>

<opening>
{}
</opening>

<eco>
{}
</eco>
"""
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
  movetext = examples["movetext"]
  white_elo = examples["white_elo"]
  black_elo = examples["black_elo"]
  white_diff = examples['white_diff']
  black_diff = examples['black_diff']
  opening = examples['opening']
  eco = examples['eco']
  texts = []

  for movetext, white_elo, black_elo, white_diff,black_diff,opening,eco in zip(movetext, white_elo, black_elo , white_diff , black_diff , opening , eco):
    text = r1_prompt.format(movetext, white_elo, black_elo , white_diff , black_diff , opening , eco)+EOS_TOKEN
    texts.append(text)

  return {"text": texts}
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df)
hf_dataset = hf_dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = hf_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Number of processors to use for processing the dataset
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2, # The batch size per GPU/TPU core
        gradient_accumulation_steps = 4, # Number of steps to perform befor each gradient accumulation
        warmup_steps = 5, # Few updates with low learning rate before actual training
        max_steps = 60, # Specifies the total number of training steps (batches) to run.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # Optimizer
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc for observability
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss
1,1.639200
2,1.471400
3,1.522600
4,1.469500
5,1.406700
6,1.302200
7,1.271000
8,1.062100
9,0.998700
10,0.894800


In [ ]:
from unsloth.chat_templates import get_chat_template
sys_prompt = """You are a strong chess engine. Given a chess position in FEN notation, predict the single best legal move for the current player in UCI format. Output only the move.
<problem>
{}
</problem>
"""
message = sys_prompt.format("""
1. b3 c5 2. Bb2 d6 3. d3 f6 4. Nd2 e5 5. c4 Bd7 6. f3 Be7 7. h3 Nh6 8. Qc2 Nf5 9. O-O-O Bf8 10. Ne4 Ne7 11. Nxd6# 1-0""")
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": message},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)# inferencing
response = tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
print(response[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

You are a strong chess engine. Given a chess position in FEN notation, predict the single best legal move for the current player in UCI format. Output only the move.
<problem>

1. b3 c5 2. Bb2 d6 3. d3 f6 4. Nd2 e5 5. c4 Bd7 6. f3 Be7 7. h3 Nh6 8. Qc2 Nf5 9. O-O-O Bf8 10. Ne4 Ne7 11. Nxd6# 1-0
</problem>
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<movetext>
1. b3 c5 2. Bb2 d6 3. d3 f6 4. Nd2 e5 5. c4 Bd7 6. f3 Nh6 7. h3 Nxg4 8. Qb1 Nxe2+ 9. Qxe2 Bxe2 10. Nxg4 Bxf3+ 11. gxf3 Bxg4 12. Bf2 Bxd1+ 13. Bxd1 0-1
</movetext>

<white_elo>
1702.0
</white_elo>

<black_elo>
1617.0
</black_elo>

<white_diff>
-5.0
</white_diff>

<black_diff>
6.0
</black_diff>

<opening>
Modern Defense: Queen's Indian Variation, Réti Variation
</opening>

<eco>
B00
</eco>
<|eot_id|>


In [ ]:
outputs

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,   5887,    220,   2366,     19,    271, 128009, 128006,
            882, 128007,    271,   2675,    527,    264,   3831,  33819,   4817,
             13,  16644,    264,  33819,   2361,    304,    435,    965,  45297,
             11,   7168,    279,   3254,   1888,   5897,   3351,    369,    279,
           1510,   2851,    304,    549,  11487,   3645,     13,   9442,   1193,
            279,   3351,    627,     27,  35686,   1363,     16,     13,    293,
             18,    272,     20,    220,     17,     13,    426,     65,     17,
            294,     21,    220,     18,     13,    294,     18,    282,     21,
            220,     19,     13,  88468,     17,    384,     20,    220,     20,
             13,    272,     19,  74660,     22,    220,     21,     13,    282,
             18,   2893,    

In [ ]:
model.save_pretrained("/content/chess-ai-merged", safe_serialization=True)
tokenizer.save_pretrained("/content/chess-ai-merged")

('/content/chess-ai-merged/tokenizer_config.json',
 '/content/chess-ai-merged/special_tokens_map.json',
 '/content/chess-ai-merged/chat_template.jinja',
 '/content/chess-ai-merged/tokenizer.json')

In [ ]:
!ls /content/chess-ai-merged


adapter_config.json	   README.md		    tokenizer.json
adapter_model.safetensors  special_tokens_map.json
chat_template.jinja	   tokenizer_config.json


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model_name = "unsloth/Llama-3.2-3B-Instruct"  # example: "Qwen/Qwen2.5-3B"
adapter_path = "/content/chess-ai"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, adapter_path)

# 🔴 THIS IS THE MOST IMPORTANT LINE
model = model.merge_and_unload()

# ✅ NOW save the REAL model
model.save_pretrained("/content/chess-ai-merged", safe_serialization=True)
tokenizer.save_pretrained("/content/chess-ai-merged")


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

('/content/chess-ai-merged/tokenizer_config.json',
 '/content/chess-ai-merged/special_tokens_map.json',
 '/content/chess-ai-merged/chat_template.jinja',
 '/content/chess-ai-merged/tokenizer.json')

In [ ]:
%cd /content/llama.cpp


/content/llama.cpp


In [ ]:
%cd /content/llama.cpp/convert


[Errno 2] No such file or directory: '/content/llama.cpp/convert'
/content


In [ ]:
!python3 convert_hf_to_gguf.py \
  --outtype f16 \
  /content/chess-ai-merged \
  --outfile /content/chess-ai.f16.gguf


INFO:hf-to-gguf:Loading model: chess-ai-merged
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00002.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00002.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {3072, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> F16, shape = {8192, 3072}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_norm.weight

In [ ]:
!ls /content | grep gguf


chess-ai.f16.gguf


In [ ]:
%cd /content/llama.cpp/build/bin
!ls

/content/llama.cpp/build/bin


In [ ]:
!./llama-quantize \
  /content/chess-ai.f16.gguf \
  /content/model-q5_k.gguf \
  q5_k

/bin/bash: line 1: ./llama-quantize: No such file or directory


In [ ]:
!find /content/llama.cpp -type f -name "*quantize*"


/content/llama.cpp/tools/quantize/quantize.cpp
/content/llama.cpp/ggml/src/ggml-cuda/quantize.cu
/content/llama.cpp/ggml/src/ggml-cuda/dequantize.cuh
/content/llama.cpp/ggml/src/ggml-cuda/quantize.cuh
/content/llama.cpp/ggml/src/ggml-vulkan/vulkan-shaders/quantize_q8_1.comp
/content/llama.cpp/ggml/src/ggml-sycl/dequantize.hpp
/content/llama.cpp/ggml/src/ggml-sycl/quantize.hpp
/content/llama.cpp/tests/test-quantize-perf.cpp
/content/llama.cpp/tests/test-quantize-fns.cpp
/content/llama.cpp/tests/test-quantize-stats.cpp
/content/llama.cpp/examples/model-conversion/scripts/utils/quantize.sh


In [ ]:
# !llama-quantize chess-ai.f16.gguf model-q5_k.gguf q5_k
!ls /content/llama.cpp/build/bin


In [ ]:
model.save_pretrained_gguf("chess-ai-001-3B-GGUF", tokenizer)

Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [02:19<00:00, 69.86s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [03:31<00:00, 105.82s/it]


Unsloth: Merge process complete. Saved to `/content/chess-ai-001-3B-GGUF`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages


RuntimeError: Unsloth: GGUF conversion failed: === Unsloth: FAILED building llama.cpp ===
Make failed: [FAIL] Command `make clean` failed with exit code 2
stdout: Makefile:6: *** Build system changed:
 The Makefile build has been replaced by CMake.

 For build instructions see:
 https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md

.  Stop.


CMake failed: [FAIL] Command `cmake . -B build -DBUILD_SHARED_LIBS=OFF -DGGML_CUDA=OFF -DLLAMA_CURL=ON` failed with error `LLAMA_CURL option is deprecated and will be ignored`
Command is deprecated!
=== Full output log: ===
Cloning into 'llama.cpp'...
Using Python 3.12.12 environment at: /usr
Resolved 27 packages in 454ms
Prepared 4 packages in 5.01s
Installed 4 packages in 38ms
 + gguf==0.17.1
 + mistral-common==1.8.8
 + pycountry==24.6.1
 + pydantic-extra-types==2.11.0
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done

In [ ]:
!ls build/bin

In [ ]:
!./quantize /content/chess-ai.gguf /content/model-q5_k.gguf q5_k


In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3)

In [ ]:
print(tokenizer._ollama_modelfile)

In [ ]:
# !ollama create unsloth_model -f ./chess-ai-001-3B-GGUF/Modelfile
!ls /content/chess-ai-001-3B-GGUF

In [ ]:
from google.colab import files

files.download("/content/chess-ai-001-3B-GGUF/model-q5_k.gguf")